In [5]:
import numpy as np
import pandas as pd

train_df = pd.read_csv('../house_data/train.csv')
test_df = pd.read_csv('../house_data/test.csv')

# Remove excluded columns
potential_excludes = ['id', 'date', 'zipcode']
cols_to_drop = [col for col in potential_excludes if col in train_df.columns]

# Separate features and target
X_train = train_df.drop(columns=cols_to_drop + ['price'])
y_train = train_df['price'].values / 1000

X_test = test_df.drop(columns=cols_to_drop + ['price'])
y_test = test_df['price'].values / 1000

# Ensure same columns
common_cols = X_train.columns.tolist()
X_test = X_test[common_cols]

print("="*70)
print("PROBLEM 5: GRADIENT DESCENT FOR LINEAR REGRESSION")
print("="*70)
print(f"\nDataset:")
print(f"  Features: {len(common_cols)}")
print(f"  Training samples: {len(y_train)}")
print(f"  Testing samples: {len(y_test)}")

# Standardize features (CRITICAL: Use same method as Problem 3)
X_train_np = X_train.values
X_test_np = X_test.values

mean = X_train_np.mean(axis=0)
std = X_train_np.std(axis=0, ddof=1)
std[std == 0] = 1.0

X_train_scaled = (X_train_np - mean) / std
X_test_scaled = (X_test_np - mean) / std

# Verify standardization
print(f"\nStandardization check:")
print(f"  Train mean: {X_train_scaled.mean():.6f} (should be ~0)")
print(f"  Train std:  {X_train_scaled.std():.6f} (should be ~1)")

# Add intercept
X_train_with_int = np.c_[np.ones(X_train_scaled.shape[0]), X_train_scaled]
X_test_with_int = np.c_[np.ones(X_test_scaled.shape[0]), X_test_scaled]

print(f"  Final shape with intercept: {X_train_with_int.shape}")

# ============================================
# HELPER FUNCTIONS
# ============================================

def predict(X, theta):
    """Make predictions using current theta"""
    return X @ theta


def compute_mse(y_true, y_pred):
    """Compute Mean Squared Error"""
    return np.mean((y_true - y_pred) ** 2)


def compute_r2(y_true, y_pred):
    """Compute R-squared score"""
    ss_res = np.sum((y_true - y_pred) ** 2)
    ss_tot = np.sum((y_true - np.mean(y_true)) ** 2)
    return 1 - (ss_res / ss_tot)


def gradient_descent(X, y, alpha, num_iterations, verbose=False):
    """
    Perform gradient descent for linear regression
    
    Cost function: J(theta) = (1/n) * sum((X@theta - y)^2)
    Gradient: dJ/d(theta) = (2/n) * X^T @ (X@theta - y)
    Update rule: theta = theta - alpha * gradient
    
    Parameters:
    -----------
    X : array (n_samples, n_features)
        Feature matrix with intercept
    y : array (n_samples,)
        Target values
    alpha : float
        Learning rate
    num_iterations : int
        Number of gradient descent iterations
    verbose : bool
        If True, print progress
    
    Returns:
    --------
    theta : array (n_features,)
        Optimized parameters
    cost_history : list
        Cost at each iteration
    """
    n_samples, n_features = X.shape
    
    # Initialize theta to zeros
    theta = np.zeros(n_features)
    
    # Track cost history
    cost_history = []
    
    for i in range(num_iterations):
        # 1. Compute predictions
        predictions = X @ theta
        
        # 2. Compute errors
        errors = predictions - y
        
        # 3. Compute gradient: (2/n) * X^T @ errors
        gradient = (2 / n_samples) * X.T @ errors
        
        # 4. Update theta
        theta = theta - alpha * gradient
        
        # 5. Track cost
        cost = np.mean(errors ** 2)
        cost_history.append(cost)
        
        # Print progress for verbose mode
        if verbose and (i == 0 or (i + 1) % 20 == 0 or i == num_iterations - 1):
            print(f"    Iteration {i+1:3d}: Cost = {cost:,.2f}")
    
    return theta, cost_history


def train_closed_form(X, y):
    """Train using closed-form solution for comparison"""
    theta = np.linalg.inv(X.T @ X) @ X.T @ y
    return theta


print("\n" + "="*70)
print("BASELINE: Results from Problem 3 (sklearn/closed-form)")
print("="*70)

train_mse_opt = 31415.75  # From Problem 3 sklearn
train_r2_opt = 0.7271     # From Problem 3 sklearn
test_mse_opt = 58834.67   # From Problem 3 sklearn  
test_r2_opt = 0.6471      # From Problem 3 sklearn

print(f"\nOptimal (Target) Solution from Problem 3:")
print(f"  Training - MSE: {train_mse_opt:>12,.2f}  |  R2: {train_r2_opt:>7.4f}")
print(f"  Testing  - MSE: {test_mse_opt:>12,.2f}  |  R2: {test_r2_opt:>7.4f}")
print("\n[OK] Using Problem 3 results as ground truth for comparison")

theta_optimal = train_closed_form(X_train_with_int, y_train)

# ============================================
# GRADIENT DESCENT EXPERIMENTS
# ============================================
print("\n" + "="*70)
print("GRADIENT DESCENT EXPERIMENTS")
print("="*70)

learning_rates = [0.01, 0.1, 0.5]
iterations_list = [10, 50, 100]

results = []

for alpha in learning_rates:
    print(f"\n{'='*70}")
    print(f"LEARNING RATE: alpha = {alpha}")
    print('='*70)
    
    for num_iter in iterations_list:
        print(f"\n  [{num_iter} iterations]")
        print(f"  {'-'*66}")
        
        # Train with gradient descent
        theta_gd, cost_history = gradient_descent(
            X_train_with_int, 
            y_train, 
            alpha=alpha, 
            num_iterations=num_iter,
            verbose=(num_iter >= 50)
        )
        
        # Check for divergence
        if np.isnan(theta_gd).any() or np.isinf(theta_gd).any():
            print(f"  [DIVERGED] Learning rate {alpha} is too large!")
            results.append({
                'Alpha': alpha,
                'Iterations': num_iter,
                'Train_MSE': np.nan,
                'Train_R2': np.nan,
                'Test_MSE': np.nan,
                'Test_R2': np.nan,
                'Theta_Distance': np.nan,
                'Status': 'DIVERGED'
            })
            continue
        
        # Make predictions
        y_train_pred_gd = predict(X_train_with_int, theta_gd)
        y_test_pred_gd = predict(X_test_with_int, theta_gd)
        
        # Compute metrics
        train_mse_gd = compute_mse(y_train, y_train_pred_gd)
        train_r2_gd = compute_r2(y_train, y_train_pred_gd)
        test_mse_gd = compute_mse(y_test, y_test_pred_gd)
        test_r2_gd = compute_r2(y_test, y_test_pred_gd)
        
        # Compare with optimal
        theta_diff = np.linalg.norm(theta_gd - theta_optimal)
        mse_diff_pct = abs(train_mse_gd - train_mse_opt) / train_mse_opt * 100
        
        # Determine status
        if mse_diff_pct < 0.1:
            status = "CONVERGED"
        elif mse_diff_pct < 1:
            status = "VERY CLOSE"
        elif mse_diff_pct < 5:
            status = "CLOSE"
        elif mse_diff_pct < 20:
            status = "PARTIAL"
        else:
            status = "NOT CONVERGED"
        
        print(f"\n  Performance:")
        print(f"    Train MSE: {train_mse_gd:>12,.2f}  |  R2: {train_r2_gd:>7.4f}")
        print(f"    Test MSE:  {test_mse_gd:>12,.2f}  |  R2: {test_r2_gd:>7.4f}")
        print(f"\n  Convergence:")
        print(f"    Distance from optimal: {theta_diff:.4f}")
        print(f"    MSE difference: {mse_diff_pct:.2f}%")
        print(f"    Status: [{status}]")
        
        # Store results
        results.append({
            'Alpha': alpha,
            'Iterations': num_iter,
            'Train_MSE': train_mse_gd,
            'Train_R2': train_r2_gd,
            'Test_MSE': test_mse_gd,
            'Test_R2': test_r2_gd,
            'Theta_Distance': theta_diff,
            'MSE_Diff_Pct': mse_diff_pct,
            'Status': status
        })


# ============================================
# SUMMARY TABLE
# ============================================
print("\n" + "="*70)
print("SUMMARY TABLE")
print("="*70)

results_df = pd.DataFrame(results)

# Display clean table
summary_cols = ['Alpha', 'Iterations', 'Train_MSE', 'Train_R2', 'Test_MSE', 'Test_R2', 'Status']
print("\n" + results_df[summary_cols].to_string(index=False))

print(f"\n{'='*70}")
print("BASELINE (Target from Closed-Form):")
print(f"  Train MSE: {train_mse_opt:>10,.2f}  |  Train R2: {train_r2_opt:>6.4f}")
print(f"  Test MSE:  {test_mse_opt:>10,.2f}  |  Test R2:  {test_r2_opt:>6.4f}")


# ============================================
# ANALYSIS
# ============================================
print("\n" + "="*70)
print("ANALYSIS")
print("="*70)

# Filter out diverged results
converged_df = results_df[results_df['Status'] != 'DIVERGED']

if len(converged_df) > 0:
    print("\n1. LEARNING RATE EFFECTS:")
    print("   " + "-"*66)
    
    for alpha in learning_rates:
        alpha_results = converged_df[converged_df['Alpha'] == alpha]
        if len(alpha_results) == 0:
            print(f"\n   alpha = {alpha}: [DIVERGED - too large!]")
            continue
        
        best_idx = alpha_results['MSE_Diff_Pct'].idxmin()
        best = alpha_results.loc[best_idx]
        
        print(f"\n   alpha = {alpha}:")
        print(f"     Best at {int(best['Iterations'])} iterations")
        print(f"     MSE difference: {best['MSE_Diff_Pct']:.2f}%")
        print(f"     Status: {best['Status']}")
    
    print("\n2. CONVERGENCE PROGRESSION:")
    print("   " + "-"*66)
    
    for alpha in learning_rates:
        alpha_results = converged_df[converged_df['Alpha'] == alpha].sort_values('Iterations')
        if len(alpha_results) == 0:
            continue
        
        print(f"\n   alpha = {alpha}:")
        for _, row in alpha_results.iterrows():
            print(f"     {int(row['Iterations']):3d} iter: MSE diff = {row['MSE_Diff_Pct']:6.2f}% [{row['Status']}]")
    
    print("\n3. BEST OVERALL CONFIGURATION:")
    print("   " + "-"*66)
    
    best_overall = converged_df.loc[converged_df['MSE_Diff_Pct'].idxmin()]
    print(f"\n   Learning rate: {best_overall['Alpha']}")
    print(f"   Iterations: {int(best_overall['Iterations'])}")
    print(f"   Train MSE: {best_overall['Train_MSE']:,.2f}")
    print(f"   Test MSE: {best_overall['Test_MSE']:,.2f}")
    print(f"   Test R2: {best_overall['Test_R2']:.4f}")
    print(f"   Difference from optimal: {best_overall['MSE_Diff_Pct']:.2f}%")

PROBLEM 5: GRADIENT DESCENT FOR LINEAR REGRESSION

Dataset:
  Features: 18
  Training samples: 1000
  Testing samples: 1000

Standardization check:
  Train mean: -0.000000 (should be ~0)
  Train std:  0.999500 (should be ~1)
  Final shape with intercept: (1000, 19)

BASELINE: Results from Problem 3 (sklearn/closed-form)

Optimal (Target) Solution from Problem 3:
  Training - MSE:    31,415.75  |  R2:  0.7271
  Testing  - MSE:    58,834.67  |  R2:  0.6471

[OK] Using Problem 3 results as ground truth for comparison

GRADIENT DESCENT EXPERIMENTS

LEARNING RATE: alpha = 0.01

  [10 iterations]
  ------------------------------------------------------------------

  Performance:
    Train MSE:   235,755.43  |  R2: -1.0476
    Test MSE:    282,905.92  |  R2: -0.6968

  Convergence:
    Distance from optimal: 787.2113
    MSE difference: 650.44%
    Status: [NOT CONVERGED]

  [50 iterations]
  ------------------------------------------------------------------
    Iteration   1: Cost = 385,968

## Problem 5: Gradient Descent Observations

### Learning Rate Effects:

- **alpha = 0.01 (Too Small):** Converges slowly. After 100 iterations, MSE = 36,766 (17% from optimal). Stable but inefficient.

- **alpha = 0.1 (Optimal):** Best performance. Reaches optimal solution (MSE = 31,416, 0.00% difference) in 100 iterations. Within 0.04% at 50 iterations.

- **alpha = 0.5 (Too Large):** Diverges completely. Cost explodes to astronomical values (10^129+). Demonstrates failure when learning rate is too large.

### Iteration Requirements:

- **10 iterations:** Insufficient. alpha = 0.1 is 11.56% from optimal.
- **50 iterations:** alpha = 0.1 converges (0.04% difference). alpha = 0.01 still needs more (122% off).
- **100 iterations:** alpha = 0.1 fully converges (0.00% difference). alpha = 0.01 improves to 17.03% but not converged.

### Convergence to Optimal:

- **Yes, gradient descent converges to the closed-form solution** with proper configuration.
- **alpha = 0.1, 100 iterations** matches sklearn exactly: Train MSE = 31,416, R^2 = 0.727; Test MSE = 58,840, R^2 = 0.647.
- Learning rate is critical: too small = slow; too large = divergence; balanced = optimal.

### Key Insights:

- Feature standardization is essential for gradient descent effectiveness
- Demonstrates bias-variance-speed tradeoff in iterative optimization
- With proper hyperparameters, iterative methods match analytical solutions